**EE782 PROJECT: CHAT BOT**\
Group members
1. Harshwardhan Waghchoure
2. Anannay Jain
3. Pradyuman


**Attention Based LSTM**

In [ ]:
import numpy as np
import codecs
import ast
import tqdm
import matplotlib.pyplot as plt
%matplotlib inline
# import seaborn as sns
import pandas as pd
import re
import tensorflow as tf
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.layers import Input, TimeDistributed, Bidirectional
from keras.models import load_model
import warnings
import nltk
nltk.download('punkt')
warnings.filterwarnings('ignore')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
# Dataset from Kaggle of customer support
rawData = pd.read_csv('/content/drive/MyDrive/ChatBotDataSet/twcs.csv')
company = "AppleSupport"                   # Taking Only AppleSupport Data
rawData.head(5)

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
0,1,sprintcare,False,Tue Oct 31 22:10:47 +0000 2017,@115712 I understand. I would like to assist y...,2,3.0
1,2,115712,True,Tue Oct 31 22:11:45 +0000 2017,@sprintcare and how do you propose we do that,NaN,1.0
2,3,115712,True,Tue Oct 31 22:08:27 +0000 2017,@sprintcare I have sent several private messag...,1,4.0
3,4,sprintcare,False,Tue Oct 31 21:54:49 +0000 2017,@115712 Please send us a Private Message so th...,3,5.0
4,5,115712,True,Tue Oct 31 21:49:35 +0000 2017,@sprintcare I did.,4,6.0


In [ ]:
# Looking at Apple company Data
rawData[rawData['author_id'].str.contains(company)].head(30)

companyAnswers = rawData[rawData['author_id'].str.contains(company)].head(1000)

companyAnswers.head(5)

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
396,696,AppleSupport,False,Tue Oct 31 22:27:49 +0000 2017,@115854 We're here for you. Which version of t...,697,698.0
398,699,AppleSupport,False,Tue Oct 31 22:36:27 +0000 2017,@115854 Lets take a closer look into this issu...,NaN,697.0
401,701,AppleSupport,False,Tue Oct 31 22:26:49 +0000 2017,@115855 Let's go to DM for the next steps. DM ...,NaN,702.0
403,703,AppleSupport,False,Tue Oct 31 22:09:52 +0000 2017,@115855 Any steps tried since it started last ...,702,704.0
405,705,AppleSupport,False,Tue Oct 31 21:57:00 +0000 2017,@115855 That's great it has iOS 11.1 as we can...,"706,704",707.0


In [ ]:
answerTweets = []
for idx, t in companyAnswers[companyAnswers['author_id'].str.contains(company)].iterrows():
     if not np.isnan(t['in_response_to_tweet_id']):

        answerTweets.append(t)

answerTweets[0]['in_response_to_tweet_id']

698.0

In [ ]:
questionTweets = []

for a in answerTweets:
    question = rawData.loc[rawData['tweet_id'] == a['in_response_to_tweet_id']]
    questionTweets.append(question['text'].to_string(index=False))



for idx, t in enumerate(answerTweets):
    answerTweets[idx] = answerTweets[idx]['text']


#Example answer and matching question
print(answerTweets[2])
print(questionTweets[2])
print(len(answerTweets))
print(len(questionTweets))

@115855 Let's go to DM for the next steps. DM us here: https://t.co/GDrqU22YpT
@AppleSupport Tried resetting my settings .. re...
997
997


In [ ]:
qListTemp = []
aListTemp = []

for t in questionTweets:
    t = re.sub('@[^\s]+','',t)
    t = re.sub('http[^\s]+','',t)
    qListTemp.append(t)

for t in answerTweets:
    t = re.sub('@[^\s]+','',t)
    t = re.sub('http[^\s]+','',t)
    aListTemp.append(t)

questionTweets = qListTemp
answerTweets = aListTemp
pairs = list(zip(questionTweets,answerTweets))

In [ ]:
df = pd.DataFrame(pairs, columns = ["question", "answer"])

In [ ]:
df.head()

,question,answer
0,,We're here for you. Which version of the iOS ...
1,The newest update. I️ made sure t...,Lets take a closer look into this issue. Sele...
2,Tried resetting my settings .. re...,Let's go to DM for the next steps. DM us here:
3,This is what it looks like,Any steps tried since it started last night?
4,I️ have an iPhone 7 Plus and yes ...,That's great it has iOS 11.1 as we can rule o...


In [ ]:
# Code refernece ChatGPT, as we didnt know how to use re.sub()
def decontractions(phrase):
    #specific
    phrase = re.sub(r"won\'t", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)
    phrase = re.sub(r"won\’t", "will not", phrase)
    phrase = re.sub(r"can\’t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    phrase = re.sub(r"n\’t", " not", phrase)
    phrase = re.sub(r"\’re", " are", phrase)
    phrase = re.sub(r"\’s", " is", phrase)
    phrase = re.sub(r"\’d", " would", phrase)
    phrase = re.sub(r"\’ll", " will", phrase)
    phrase = re.sub(r"\’t", " not", phrase)
    phrase = re.sub(r"\’ve", " have", phrase)
    phrase = re.sub(r"\’m", " am", phrase)

    return phrase

def preprocess(text):
    text = text.lower()
    text = decontractions(text)
    text = re.sub('[^A-Za-z0-9 ]+', '', text)
    return text


df['question'] = df['question'].apply(preprocess)
df['answer'] = df['answer'].apply(preprocess)
df.head()

,question,answer
0,,we are here for you which version of the ios ...
1,the newest update i made sure t,lets take a closer look into this issue selec...
2,tried resetting my settings re,let is go to dm for the next steps dm us here
3,this is what it looks like,any steps tried since it started last night
4,i have an iphone 7 plus and yes,that is great it has ios 111 as we can rule o...


In [ ]:
short_questions_tok = [nltk.word_tokenize(sent) for sent in df["question"]]
short_answers_tok = [nltk.word_tokenize(sent) for sent in df["answer"]]

In [ ]:
#train-validation split
data_size = len(short_questions_tok)

# We will use the first 0-80th %-tile (80%) of data for the training
training_input  = short_questions_tok[:round(data_size*(80/100))]
training_input  = [tr_input[::-1] for tr_input in training_input] #reverseing input seq for better performance
training_output = short_answers_tok[:round(data_size*(80/100))]

# We will use the remaining for validation
validation_input = short_questions_tok[round(data_size*(80/100)):]
validation_input  = [val_input[::-1] for val_input in validation_input] #reverseing input seq for better performance
validation_output = short_answers_tok[round(data_size*(80/100)):]

print('training size', len(training_input))
print('validation size', len(validation_input))

training size 798
validation size 199


In [ ]:
# Create a dictionary for the frequency of the vocabulary
# Create
vocab = {}
for question in short_questions_tok:
    for word in question:
        if word not in vocab:
            vocab[word] = 1
        else:
            vocab[word] += 1

for answer in short_answers_tok:
    for word in answer:
        if word not in vocab:
            vocab[word] = 1
        else:
            vocab[word] += 1

In [ ]:
# Remove rare words from the vocabulary.
# We will aim to replace fewer than 5% of words with <UNK>
# You will see this ratio soon.
threshold = 1
count = 0
for k,v in vocab.items():
    if v >= threshold:
        count += 1

In [ ]:
print("Size of total vocab:", len(vocab))
print("Size of vocab we will use:", count)

Size of total vocab: 2011
Size of vocab we will use: 2011


In [ ]:
#we will create dictionaries to provide a unique integer for each word.
WORD_CODE_START = 1
WORD_CODE_PADDING = 0


word_num  = 2 #number 1 is left for WORD_CODE_START for model decoder later
encoding = {}
decoding = {1: 'START'}
for word, count in vocab.items():
    if count >= threshold: #get vocabularies that appear above threshold count
        encoding[word] = word_num
        decoding[word_num] = word
        word_num += 1

print("No. of vocab used:", word_num)

No. of vocab used: 2013


In [ ]:
#include unknown token for words not in dictionary
decoding[len(encoding)+2] = '<UNK>'
encoding['<UNK>'] = len(encoding)+2

In [ ]:
dict_size = word_num+1
dict_size

2014

In [ ]:
def transform(encoding, data, vector_size=20):
    """
    :param encoding: encoding dict built by build_word_encoding()
    :param data: list of strings
    :param vector_size: size of each encoded vector
    """
    transformed_data = np.zeros(shape=(len(data), vector_size))
    for i in range(len(data)):
        for j in range(min(len(data[i]), vector_size)):
            try:
                transformed_data[i][j] = encoding[data[i][j]]
            except:
                transformed_data[i][j] = encoding['<UNK>']
    return transformed_data

In [ ]:
#encoding training set
encoded_training_input = transform(
    encoding, training_input, vector_size=20)
encoded_training_output = transform(
    encoding, training_output, vector_size=20)

print('encoded_training_input', encoded_training_input.shape)
print('encoded_training_output', encoded_training_output.shape)

encoded_training_input (798, 20)
encoded_training_output (798, 20)


In [ ]:
#encoding validation set
encoded_validation_input = transform(
    encoding, validation_input, vector_size=20)
encoded_validation_output = transform(
    encoding, validation_output, vector_size=20)

print('encoded_validation_input', encoded_validation_input.shape)
print('encoded_validation_output', encoded_validation_output.shape)

encoded_validation_input (199, 20)
encoded_validation_output (199, 20)


In [ ]:
import tensorflow as tf
tf.keras.backend.clear_session()

In [ ]:
INPUT_LENGTH = 20
OUTPUT_LENGTH = 20

encoder_input = Input(shape=(INPUT_LENGTH,))
decoder_input = Input(shape=(OUTPUT_LENGTH,))

In [ ]:
from keras.layers import SimpleRNN

encoder = Embedding(dict_size, 128, input_length=INPUT_LENGTH, mask_zero=True)(encoder_input)
encoder = LSTM(512, return_sequences=True, unroll=True)(encoder)
encoder_last = encoder[:,-1,:]

print('encoder', encoder)
print('encoder_last', encoder_last)

decoder = Embedding(dict_size, 128, input_length=OUTPUT_LENGTH, mask_zero=True)(decoder_input)
decoder = LSTM(512, return_sequences=True, unroll=True)(decoder, initial_state=[encoder_last, encoder_last])

print('decoder', decoder)

# For the plain Sequence-to-Sequence, we produced the output from directly from decoder
# output = TimeDistributed(Dense(output_dict_size, activation="softmax"))(decoder)

encoder KerasTensor(type_spec=TensorSpec(shape=(None, 20, 512), dtype=tf.float32, name=None), name='lstm/transpose_2:0', description="created by layer 'lstm'")
encoder_last KerasTensor(type_spec=TensorSpec(shape=(None, 512), dtype=tf.float32, name=None), name='tf.__operators__.getitem/strided_slice:0', description="created by layer 'tf.__operators__.getitem'")
decoder KerasTensor(type_spec=TensorSpec(shape=(None, 20, 512), dtype=tf.float32, name=None), name='lstm_1/transpose_2:0', description="created by layer 'lstm_1'")


In [ ]:
from keras.layers import Activation, dot, concatenate

# Equation (7) with 'dot' score from Section 3.1 in the paper.
# Note that we reuse Softmax-activation layer instead of writing tensor calculation
attention = dot([decoder, encoder], axes=[2, 2])
attention = Activation('softmax', name='attention')(attention)
print('attention', attention)

context = dot([attention, encoder], axes=[2,1])
print('context', context)

decoder_combined_context = concatenate([context, decoder])
print('decoder_combined_context', decoder_combined_context)

# Has another weight + tanh layer
output = TimeDistributed(Dense(512, activation="tanh"))(decoder_combined_context)
output = TimeDistributed(Dense(dict_size, activation="softmax"))(output)
print('output', output)

attention KerasTensor(type_spec=TensorSpec(shape=(None, 20, 20), dtype=tf.float32, name=None), name='attention/Softmax:0', description="created by layer 'attention'")
context KerasTensor(type_spec=TensorSpec(shape=(None, 20, 512), dtype=tf.float32, name=None), name='dot_1/MatMul:0', description="created by layer 'dot_1'")
decoder_combined_context KerasTensor(type_spec=TensorSpec(shape=(None, 20, 1024), dtype=tf.float32, name=None), name='concatenate/concat:0', description="created by layer 'concatenate'")
output KerasTensor(type_spec=TensorSpec(shape=(None, 20, 2014), dtype=tf.float32, name=None), name='time_distributed_1/Reshape_1:0', description="created by layer 'time_distributed_1'")


In [ ]:
model = Model(inputs=[encoder_input, decoder_input], outputs=[output])
model.compile(optimizer='adam', loss='binary_crossentropy')
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 20)]                 0         []                            
                                                                                                  
 embedding (Embedding)       (None, 20, 128)              257792    ['input_1[0][0]']             
                                                                                                  
 input_2 (InputLayer)        [(None, 20)]                 0         []                            
                                                                                                  
 lstm (LSTM)                 (None, 20, 512)              1312768   ['embedding[0][0]']           
                                                                                              

In [ ]:
training_encoder_input = encoded_training_input
training_decoder_input = np.zeros_like(encoded_training_output)
training_decoder_input[:, 1:] = encoded_training_output[:,:-1]
training_decoder_input[:, 0] = WORD_CODE_START
training_decoder_output = np.eye(dict_size)[encoded_training_output.astype('int')]

validation_encoder_input = encoded_validation_input
validation_decoder_input = np.zeros_like(encoded_validation_output)
validation_decoder_input[:, 1:] = encoded_validation_output[:,:-1]
validation_decoder_input[:, 0] = WORD_CODE_START
validation_decoder_output = np.eye(dict_size)[encoded_validation_output.astype('int')]

In [ ]:
model.fit(x=[training_encoder_input, training_decoder_input], y=[training_decoder_output],
          validation_data=([validation_encoder_input, validation_decoder_input], [validation_decoder_output]),
          #validation_split=0.05,
          batch_size=64, epochs=100)

model.save('model_attention.h5')

Epoch 1/100
13/13 [==============================] - 21s 304ms/step - loss: 0.0043 - val_loss: 0.0043
Epoch 2/100
13/13 [==============================] - 2s 191ms/step - loss: 0.0042 - val_loss: 0.0042
Epoch 3/100
13/13 [==============================] - 3s 231ms/step - loss: 0.0042 - val_loss: 0.0041
Epoch 4/100
13/13 [==============================] - 3s 218ms/step - loss: 0.0037 - val_loss: 0.0033
Epoch 5/100
13/13 [==============================] - 1s 81ms/step - loss: 0.0032 - val_loss: 0.0031
Epoch 6/100
13/13 [==============================] - 1s 84ms/step - loss: 0.0031 - val_loss: 0.0031
Epoch 7/100
13/13 [==============================] - 1s 90ms/step - loss: 0.0030 - val_loss: 0.0030
Epoch 8/100
13/13 [==============================] - 1s 84ms/step - loss: 0.0030 - val_loss: 0.0030
Epoch 9/100
13/13 [==============================] - 1s 76ms/step - loss: 0.0030 - val_loss: 0.0030
Epoch 10/100
13/13 [==============================] - 1s 77ms/step - loss: 0.0029 - val_loss: 0

In [ ]:
def prediction(raw_input):
    clean_input = decontractions(raw_input)
    input_tok = [nltk.word_tokenize(clean_input)]
    input_tok = [input_tok[0][::-1]]  #reverseing input seq
    encoder_input = transform(encoding, input_tok, 20)
    decoder_input = np.zeros(shape=(len(encoder_input), OUTPUT_LENGTH))
    decoder_input[:,0] = WORD_CODE_START
    for i in range(1, OUTPUT_LENGTH):
        output = model.predict([encoder_input, decoder_input]).argmax(axis=2)
        decoder_input[:,i] = output[:,i]
    return output

def decode(decoding, vector):
    """
    :param decoding: decoding dict built by word encoding
    :param vector: an encoded vector
    """
    text = ''
    for i in vector:
        if i == 0:
            break
        text += ' '
        text += decoding[i]
    return text

In [ ]:
df["question"][np.random.randint(1, len(df["question"]))]

'this what happens to my iphone after i download'

In [ ]:
for i in range(20):
    seq_index = np.random.randint(1, len(df["question"]))
    output = prediction(df["question"][seq_index])
    print ('Q:', df["question"][seq_index])
    print ('A:', decode(decoding, output[0]))

1/1 [==============================] - 0s 25ms/step
Q:  how long does it take usually for
A:  we would like to help with your iphone too smoothly and which ios version and ios version are you you
1/1 [==============================] - 0s 22ms/step
Q:  no
A:  we offer support via twitter in english contact us for help in your preferred language here
1/1 [==============================] - 0s 22ms/step
Q:  it is still the same
A:  we would like to help with your iphone let is start on a dm us in dm
1/1 [==============================] - 0s 24ms/step
Q: after the last update i cannot hear my phone c
A:  we would like to help with your iphone and let us know the ios version and we will continue continue
1/1 [==============================] - 0s 22ms/step
Q:  thank you
A:  you are welcome to ios 111 if you need to see if the issue continues the update
1/1 [==============================] - 0s 34ms/step
Q:  why can not i log into my icloud on
A:  we would like to help with your iphone of ios 